In [4]:
# !pip install transformers

In [5]:
#!pip install notebook --upgrade
#!pip install jupyter --upgrade
#!pip install accelerate

In [6]:
import torch
print(torch.version.cuda)
print(torch.cuda.is_available())

12.4
True


In [11]:
# Use a pipeline as a high-level helper
from transformers import pipeline

messages = [
    {"role": "user", "content": "Who are you?"},
]
pipe = pipeline("text-generation", model="LGAI-EXAONE/EXAONE-3.5-2.4B-Instruct", trust_remote_code=True, device=0)

loading configuration file config.json from cache at C:\Users\b1005\.cache\huggingface\hub\models--LGAI-EXAONE--EXAONE-3.5-2.4B-Instruct\snapshots\8e6fc27d1910b526b5d48a2aa129b08a0293df5e\config.json
loading configuration file config.json from cache at C:\Users\b1005\.cache\huggingface\hub\models--LGAI-EXAONE--EXAONE-3.5-2.4B-Instruct\snapshots\8e6fc27d1910b526b5d48a2aa129b08a0293df5e\config.json
Model config ExaoneConfig {
  "_name_or_path": "LGAI-EXAONE/EXAONE-3.5-2.4B-Instruct",
  "activation_function": "silu",
  "architectures": [
    "ExaoneForCausalLM"
  ],
  "attention_dropout": 0.0,
  "auto_map": {
    "AutoConfig": "LGAI-EXAONE/EXAONE-3.5-2.4B-Instruct--configuration_exaone.ExaoneConfig",
    "AutoModelForCausalLM": "LGAI-EXAONE/EXAONE-3.5-2.4B-Instruct--modeling_exaone.ExaoneForCausalLM",
    "AutoModelForSequenceClassification": "LGAI-EXAONE/EXAONE-3.5-2.4B-Instruct--modeling_exaone.ExaoneForSequenceClassification"
  },
  "bos_token_id": 1,
  "embed_dropout": 0.0,
  "eos_tok

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

All model checkpoint weights were used when initializing ExaoneForCausalLM.

All the weights of ExaoneForCausalLM were initialized from the model checkpoint at LGAI-EXAONE/EXAONE-3.5-2.4B-Instruct.
If your task is similar to the task the model of the checkpoint was trained on, you can already use ExaoneForCausalLM for predictions without further training.
loading configuration file generation_config.json from cache at C:\Users\b1005\.cache\huggingface\hub\models--LGAI-EXAONE--EXAONE-3.5-2.4B-Instruct\snapshots\8e6fc27d1910b526b5d48a2aa129b08a0293df5e\generation_config.json
Generate config GenerationConfig {
  "bos_token_id": 1,
  "eos_token_id": 361,
  "pad_token_id": 0
}



In [12]:
from transformers import AutoModelForCausalLM
model = AutoModelForCausalLM.from_pretrained("LGAI-EXAONE/EXAONE-3.5-2.4B-Instruct", trust_remote_code=True)

loading configuration file config.json from cache at C:\Users\b1005\.cache\huggingface\hub\models--LGAI-EXAONE--EXAONE-3.5-2.4B-Instruct\snapshots\8e6fc27d1910b526b5d48a2aa129b08a0293df5e\config.json
loading configuration file config.json from cache at C:\Users\b1005\.cache\huggingface\hub\models--LGAI-EXAONE--EXAONE-3.5-2.4B-Instruct\snapshots\8e6fc27d1910b526b5d48a2aa129b08a0293df5e\config.json
Model config ExaoneConfig {
  "_name_or_path": "LGAI-EXAONE/EXAONE-3.5-2.4B-Instruct",
  "activation_function": "silu",
  "architectures": [
    "ExaoneForCausalLM"
  ],
  "attention_dropout": 0.0,
  "auto_map": {
    "AutoConfig": "LGAI-EXAONE/EXAONE-3.5-2.4B-Instruct--configuration_exaone.ExaoneConfig",
    "AutoModelForCausalLM": "LGAI-EXAONE/EXAONE-3.5-2.4B-Instruct--modeling_exaone.ExaoneForCausalLM",
    "AutoModelForSequenceClassification": "LGAI-EXAONE/EXAONE-3.5-2.4B-Instruct--modeling_exaone.ExaoneForSequenceClassification"
  },
  "bos_token_id": 1,
  "embed_dropout": 0.0,
  "eos_tok

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

All model checkpoint weights were used when initializing ExaoneForCausalLM.

All the weights of ExaoneForCausalLM were initialized from the model checkpoint at LGAI-EXAONE/EXAONE-3.5-2.4B-Instruct.
If your task is similar to the task the model of the checkpoint was trained on, you can already use ExaoneForCausalLM for predictions without further training.
loading configuration file generation_config.json from cache at C:\Users\b1005\.cache\huggingface\hub\models--LGAI-EXAONE--EXAONE-3.5-2.4B-Instruct\snapshots\8e6fc27d1910b526b5d48a2aa129b08a0293df5e\generation_config.json
Generate config GenerationConfig {
  "bos_token_id": 1,
  "eos_token_id": 361,
  "pad_token_id": 0
}



In [13]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

model_name = "LGAI-EXAONE/EXAONE-3.5-2.4B-Instruct"

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.bfloat16,
    trust_remote_code=True,
    device_map="auto"
)
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Choose your prompt
prompt = "스스로를 자랑해 봐"       # Korean example

messages = [
    {"role": "system", 
     "content": "You are EXAONE model from LG AI Research, a helpful assistant."},
    {"role": "user", "content": prompt}
]
input_ids = tokenizer.apply_chat_template(
    messages,
    tokenize=True,
    add_generation_prompt=True,
    return_tensors="pt"
)

output = model.generate(
    input_ids.to("cuda"),
    eos_token_id=tokenizer.eos_token_id,
    max_new_tokens=128,
    do_sample=False,
)
print(tokenizer.decode(output[0]))

loading configuration file config.json from cache at C:\Users\b1005\.cache\huggingface\hub\models--LGAI-EXAONE--EXAONE-3.5-2.4B-Instruct\snapshots\8e6fc27d1910b526b5d48a2aa129b08a0293df5e\config.json
loading configuration file config.json from cache at C:\Users\b1005\.cache\huggingface\hub\models--LGAI-EXAONE--EXAONE-3.5-2.4B-Instruct\snapshots\8e6fc27d1910b526b5d48a2aa129b08a0293df5e\config.json
Model config ExaoneConfig {
  "_name_or_path": "LGAI-EXAONE/EXAONE-3.5-2.4B-Instruct",
  "activation_function": "silu",
  "architectures": [
    "ExaoneForCausalLM"
  ],
  "attention_dropout": 0.0,
  "auto_map": {
    "AutoConfig": "LGAI-EXAONE/EXAONE-3.5-2.4B-Instruct--configuration_exaone.ExaoneConfig",
    "AutoModelForCausalLM": "LGAI-EXAONE/EXAONE-3.5-2.4B-Instruct--modeling_exaone.ExaoneForCausalLM",
    "AutoModelForSequenceClassification": "LGAI-EXAONE/EXAONE-3.5-2.4B-Instruct--modeling_exaone.ExaoneForSequenceClassification"
  },
  "bos_token_id": 1,
  "embed_dropout": 0.0,
  "eos_tok

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

All model checkpoint weights were used when initializing ExaoneForCausalLM.

All the weights of ExaoneForCausalLM were initialized from the model checkpoint at LGAI-EXAONE/EXAONE-3.5-2.4B-Instruct.
If your task is similar to the task the model of the checkpoint was trained on, you can already use ExaoneForCausalLM for predictions without further training.
loading configuration file generation_config.json from cache at C:\Users\b1005\.cache\huggingface\hub\models--LGAI-EXAONE--EXAONE-3.5-2.4B-Instruct\snapshots\8e6fc27d1910b526b5d48a2aa129b08a0293df5e\generation_config.json
Generate config GenerationConfig {
  "bos_token_id": 1,
  "eos_token_id": 361,
  "pad_token_id": 0
}

Some parameters are on the meta device because they were offloaded to the cpu.
loading file vocab.json from cache at C:\Users\b1005\.cache\huggingface\hub\models--LGAI-EXAONE--EXAONE-3.5-2.4B-Instruct\snapshots\8e6fc27d1910b526b5d48a2aa129b08a0293df5e\vocab.json
loading file merges.txt from cache at C:\Users\b1005\.c

[|system|]You are EXAONE model from LG AI Research, a helpful assistant.[|endofturn|]
[|user|]스스로를 자랑해 봐
[|assistant|]저는 LG AI Research에서 개발된 EXAONE 모델로서, 뛰어난 자연어 처리 능력을 바탕으로 다양한 언어 작업을 수행할 수 있습니다. 지속적인 학습을 통해 사용자의 질문에 정확하고 신속하게 응답하며, 복잡한 정보도 명확하게 전달하는 데 초점을 맞춥니다. 이러한 기술 덕분에 다양한 분야에서 효율적인 지원을 제공하며, 사용자 경험 향상에 기여하고 있습니다.[|endofturn|]


In [10]:
from transformers import logging
logging.set_verbosity_info()

In [20]:
# Choose your prompt
prompt = input()       # Korean example

messages = [
    {"role": "system", 
     "content": "You are EXAONE model from LG AI Research, a helpful assistant."},
    {"role": "user", "content": prompt}
]
input_ids = tokenizer.apply_chat_template(
    messages,
    tokenize=True,
    add_generation_prompt=True,
    return_tensors="pt"
)

output = model.generate(
    input_ids.to("cuda"),
    eos_token_id=tokenizer.eos_token_id,
    max_new_tokens=128,
    do_sample=False,
)
print(tokenizer.decode(output[0]))

 너는 어떠한 역할을 수행할 수 있어?


[|system|]You are EXAONE model from LG AI Research, a helpful assistant.[|endofturn|]
[|user|]너는 어떠한 역할을 수행할 수 있어?
[|assistant|]저는 LG AI Research에서 개발된 EXAONE 모델로서 다양한 역할을 수행할 수 있습니다:

1. **텍스트 생성 및 이해**: 자연스러운 텍스트 생성과 복잡한 언어 이해를 통해 다양한 질문에 답변하거나 지시사항을 처리합니다.
2. **대화 지원**: 사용자와 자연스러운 대화를 통해 정보 제공, 문제 해결, 일상적인 질문에 대한 답변 등을 지원합니다.
3. **지식 검색**: 제공된 정보나 질문에 대한 답변을 통해 필요한 지식을 검색하고 제공합니다.
4. **코드 생성 및 수정**
